In [41]:
import os
import re
import math
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
!pip -q install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 67.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [24]:
import nltk

from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
!wget https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/HwrqIBfc4gXRMQ/food-dataset-ru.csv

--2023-06-30 19:42:25--  https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/HwrqIBfc4gXRMQ/food-dataset-ru.csv
Resolving getfile.dokpub.com (getfile.dokpub.com)... 142.132.255.217
Connecting to getfile.dokpub.com (getfile.dokpub.com)|142.132.255.217|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloader.disk.yandex.ru/disk/99a372837a20d9583aacf999cca04c66eab78d33f0c983fdc012b0b19f0af159/649f6861/xw5Ld4ahZ3PXthbshI9L6XH8J5jyn8ebkeiO1sv7L-A_zWAfaXOCgzOjU9saaKkKBMf3W20PsRL97S6mz1dQFA%3D%3D?uid=0&filename=food-dataset-ru.csv&disposition=attachment&hash=jwBgKhoRZnhy3Vo2lwXI7M/NsSuFKWrDxrJ47U6qkrLZlz61neDDkjC/X/Kd2QG2q/J6bpmRyOJonT3VoXnDag%3D%3D%3A/food-dataset-ru.csv&limit=0&content_type=text%2Fplain&owner_uid=1632007365&fsize=66912433&hid=867f6ecc62376fca5dcaad437eaf35c1&media_type=spreadsheet&tknv=v2 [following]
--2023-06-30 19:42:25--  https://downloader.disk.yandex.ru/disk/99a372837a20d9583aacf999cca04c66eab78d33f0c983fdc012b0b19f0a

In [5]:
df_items = pd.read_csv('food-dataset-ru.csv')

In [6]:
df_items.head()

,name,text,ingredient,energy,time_cook,type_kitchen,link,label
0,Завтрак для ленивых,"В широкую миску выложить творог, добавить яйцо...","Куриное яйцо: 1 штука, Мягкий творог: 200 г, П...","Калорий 87 ккал, Белки 8 грамм, Жиры 2 грамм, ...",Русская кухня,15 минут,https://eda.ru/recepty/zavtraki/zavtrak-dlja-l...,Завтрак
1,Гречневый завтрак,"Гречку промыть, залить 2 стаканами кипятка, по...","Гречневая крупа: 1 стакан, Рубленая петрушка: ...","Калорий 284 ккал, Белки 6 грамм, Жиры 11 грамм...",Русская кухня,1 час 20 минут,https://eda.ru/recepty/zavtraki/grechnevij-zav...,Завтрак
2,Завтрак детства,Морковь и зеленое яблоко натереть на средней т...,"Морковь: 1 штука, Яблоко: 1 штука, Апельсины: ...","Калорий 623 ккал, Белки 13 грамм, Жиры 22 грам...",Русская кухня,5 минут,https://eda.ru/recepty/zavtraki/zavtrak-detstv...,Завтрак
3,Полный английский завтрак,"Разогреть духовку до 180 градусов, а сковороду...","Куриное яйцо: 2 штуки, Свиные сосиски: 2 штуки...","Калорий 907 ккал, Белки 26 грамм, Жиры 75 грам...",Британская кухня,30 минут,https://eda.ru/recepty/zavtraki/polnyy-angliys...,Завтрак
4,Французские гренки к завтраку,Смешать яйцо с молоком. Посолить. Обмакнуть ку...,"Батон: 3 куска, Молоко: 2 столовые ложки, Кури...","Калорий 519 ккал, Белки 16 грамм, Жиры 29 грам...",Французская кухня,10 минут,https://eda.ru/recepty/zavtraki/francuzskie-gr...,Завтрак


In [29]:
def preprocessing(df_items):
    text_list = []
    stopwords_ru = stopwords.words("russian")
    mystem = Mystem()
    for i in range(df_items.shape[0]):
        text = " ".join(df_items.iloc[i, :5].values)
        text = text.lower()
        text = re.sub(r'\([^()]*\)', '', text)
        text = re.sub(r'[^а-яё]', ' ', text)
        text = re.sub(" +", " ", text)

        tokens = mystem.lemmatize(text)
        text = " ".join([token for token in tokens\
                         if not token in stopwords_ru and token != " " and len(token) > 1])

        if i % 500 == 0:
            print(f"{i} / {df_items.shape[0]}")
        text_list.append(text)

    return text_list

In [30]:
text_list = preprocessing(df_items)

0 / 37638
500 / 37638
1000 / 37638
1500 / 37638
2000 / 37638
2500 / 37638
3000 / 37638
3500 / 37638
4000 / 37638
4500 / 37638
5000 / 37638
5500 / 37638
6000 / 37638
6500 / 37638
7000 / 37638
7500 / 37638
8000 / 37638
8500 / 37638
9000 / 37638
9500 / 37638
10000 / 37638
10500 / 37638
11000 / 37638
11500 / 37638
12000 / 37638
12500 / 37638
13000 / 37638
13500 / 37638
14000 / 37638
14500 / 37638
15000 / 37638
15500 / 37638
16000 / 37638
16500 / 37638
17000 / 37638
17500 / 37638
18000 / 37638
18500 / 37638
19000 / 37638
19500 / 37638
20000 / 37638
20500 / 37638
21000 / 37638
21500 / 37638
22000 / 37638
22500 / 37638
23000 / 37638
23500 / 37638
24000 / 37638
24500 / 37638
25000 / 37638
25500 / 37638
26000 / 37638
26500 / 37638
27000 / 37638
27500 / 37638
28000 / 37638
28500 / 37638
29000 / 37638
29500 / 37638
30000 / 37638
30500 / 37638
31000 / 37638
31500 / 37638
32000 / 37638
32500 / 37638
33000 / 37638
33500 / 37638
34000 / 37638
34500 / 37638
35000 / 37638
35500 / 37638
36000 / 37638
36

In [31]:
text_list[2]

'завтрак детство морковь зеленый яблоко натирать средний терка апельсин снимать цедра половинка выдавливать сок второй нарезать небольшой кусочек смешать яблоко морковь апельсин изюм орех заправлять сок мед добавлять корица морковь штука яблоко штука апельсин штука изюм мед чайный ложка орех молотый корица вкус калория ккал белок грамм жир грамм углевод грамм русский кухня'

In [44]:
vectorizer = TfidfVectorizer()
items_vec = vectorizer.fit_transform(text_list)

In [54]:
def get_recommendation(text, vectorizer, items_vec):
    stopwords_ru = stopwords.words("russian")
    mystem = Mystem()

    text = text.lower()
    text = re.sub(r'\([^()]*\)', '', text)
    text = re.sub(r'[^а-яё]', ' ', text)
    text = re.sub(" +", " ", text)

    tokens = mystem.lemmatize(text)
    text = " ".join([token for token in tokens\
                    if not token in stopwords_ru and token != " " and len(token) > 1])
    text = vectorizer.transform([text])

    rating = cosine_similarity(items_vec, text).reshape(-1)
    rating_arg = np.argsort(rating)[::-1]

    return rating_arg.tolist()

Первая рекомендация

In [58]:
text = " ".join(df_items.iloc[34529, :5].values)
rating = get_recommendation(text, vectorizer, items_vec)
df_items.iloc[rating].head()

,name,text,ingredient,energy,time_cook,type_kitchen,link,label
34529,Разноцветные американские кукис,Разогрейте духовку до 180 градусов. Сливочное ...,"Сливочное масло: 120 г, Ванилин: 1 чайная ложк...","Калорий 2672 ккал, Белки 32 грамм, Жиры 134 гр...",Американская кухня,1 час,https://eda.ru/recepty/vypechka-deserty/raznoc...,Выпечка
30453,Печенье с шоколадным драже,Смешать в миске размягченное масло и сахар до ...,"Сливочное масло: 200 г, Тростниковый сахар: 1 ...","Калорий 366 ккал, Белки 5 грамм, Жиры 20 грамм...",Европейская кухня,25 минут,https://eda.ru/recepty/vypechka-deserty/pechen...,Выпечка
35946,Овсяное печенье с шоколадом и фундуком,Разогрейте духовку до 180 градусов. Возьмите м...,"Овсяная мука: 150 г, Овсяные хлопья: 60 г, Сол...","Калорий 3484 ккал, Белки 54 грамм, Жиры 209 гр...",Европейская кухня,1 час,https://eda.ru/recepty/vypechka-deserty/ovsjan...,Выпечка
34673,Печенье с тремя видами шоколада,Разогреть духовку до 180 градусов. Смазать про...,"Сливочное масло: 80 г, Тростниковый сахар: 180...","Калорий 2816 ккал, Белки 35 грамм, Жиры 129 гр...",Европейская кухня,1 час,https://eda.ru/recepty/vypechka-deserty/pechen...,Выпечка
34497,Домашнее пряное овсяное печенье,Нагреть духовку до 180 градусов. Застелить про...,"Пшеничная мука: 245 г, Овсяные хлопья: 300 г, ...","Калорий 2581 ккал, Белки 39 грамм, Жиры 113 гр...",Европейская кухня,40 минут,https://eda.ru/recepty/vypechka-deserty/domash...,Выпечка


Вторая рекомендация

In [59]:
text = " ".join(df_items.iloc[12621, :5].values)
rating = get_recommendation(text, vectorizer, items_vec)
df_items.iloc[rating].head()

,name,text,ingredient,energy,time_cook,type_kitchen,link,label
12621,Говядина в пяти специях,"Куриные и свиные кости залить кипятком (5 л), ...","Куриные кости : 1 кг, Свиные кости: 1 кг, Имби...","Калорий 1178 ккал, Белки 68 грамм, Жиры 82 гра...",Европейская кухня,45 минут,https://eda.ru/recepty/osnovnye-blyuda/govjadi...,Основное блюдо
25303,Маринад с пятью специями и хересом,Подготовить все ингредиенты. Поместить в миску...,"Соевый соус: ¼ стакана, Сухой херес: ¼ стакана...","Калорий 152 ккал, Белки 2 грамм, Жиры 12 грамм...",Европейская кухня,30 минут,https://eda.ru/recepty/sousy-marinady/marinad-...,Соус
27728,"Щи с телятиной, курицей и лимонником",Телячьи кости и куриные крылья залить пятью ли...,"Телячьи кости: 1 кг, Куриные крылья: 1 кг, Бел...","Калорий 600 ккал, Белки 57 грамм, Жиры 25 грам...",Русская кухня,3 часа,https://eda.ru/recepty/supy/schi-s-teljatinoj-...,Суп
10626,Говядина в соусе «Пять специй»,"Измельченные чеснок, лук, чили и тертый имбирь...","Оливковое масло: 65 мл, Чеснок: 6 зубчиков, Пе...","Калорий 642 ккал, Белки 51 грамм, Жиры 42 грам...",Китайская кухня,2 часа,https://eda.ru/recepty/osnovnye-blyuda/govjadi...,Основное блюдо
12991,Копченая рыба,Филе трески замариновать на полчаса в смеси со...,"Филе трески: 600 г, Соевый соус: 30 мл, Херес:...","Калорий 892 ккал, Белки 19 грамм, Жиры 84 грам...",Европейская кухня,1 час,https://eda.ru/recepty/osnovnye-blyuda/kopchen...,Основное блюдо


Третья рекомендация

In [60]:
text = " ".join(df_items.iloc[6177, :5].values)
rating = get_recommendation(text, vectorizer, items_vec)
df_items.iloc[rating].head()

,name,text,ingredient,energy,time_cook,type_kitchen,link,label
6177,Форшмак «Дружба народов»,"Почистить селедку, или купить готовое филе в м...","Морковь: 1 штука, Плавленый сырок: 3 штуки, Фи...","Калорий 277 ккал, Белки 21 грамм, Жиры 14 грам...",Европейская кухня,20 минут,https://eda.ru/recepty/zakuski/forshmak-druzhb...,Закуска
5927,Масло из селедки,"Взять сельдь. Очистить от плавников, головы, к...","Сливочное масло: 200 г, Слабосоленая сельдь: 4...","Калорий 1087 ккал, Белки 59 грамм, Жиры 85 гра...",Русская кухня,31 минута,https://eda.ru/recepty/zakuski/maslo-iz-seledk...,Закуска
25975,Суп с шампиньонами и плавленым сыром,Нарезать шампиньоны и лук. Добавить сметану. О...,"Шампиньоны: 200 г, Курица: 300 г, Картофель: 2...","Калорий 300 ккал, Белки 20 грамм, Жиры 16 грам...",Французская кухня,1 час,https://eda.ru/recepty/supy/sup-s-shampinonami...,Суп
25696,Суп из плавленых сырков,"Вскипятить в кастрюле 2 л воды, добавить очище...","Корень сельдерея: 1 штука, Плавленый сырок: 2 ...","Калорий 405 ккал, Белки 20 грамм, Жиры 18 грам...",Французская кухня,25 минут,https://eda.ru/recepty/supy/sup-iz-plavlenih-s...,Суп
12162,Куриные рубленые котлеты с плавленым сыром,Куриное филе нарезать средними кубиками. Морко...,"Куриное филе: 500 г, Плавленый сырок: 2 штуки,...","Калорий 767 ккал, Белки 50 грамм, Жиры 48 грам...",Европейская кухня,30 минут,https://eda.ru/recepty/osnovnye-blyuda/kurinie...,Основное блюдо


Четвертая рекомендация

In [61]:
text = "Ризотто"
rating = get_recommendation(text, vectorizer, items_vec)
df_items.iloc[rating].head()

,name,text,ingredient,energy,time_cook,type_kitchen,link,label
20125,Яблочное ризотто с беконом,Очистить и нарезать лук мелкими дольками. Наре...,"Рис для ризотто: 150 г, Яблочный сок: 500 мл, ...","Калорий 626 ккал, Белки 12 грамм, Жиры 40 грам...",Итальянская кухня,1 час,https://eda.ru/recepty/rizotto/yablochnoe-rizo...,Ризотто
20101,Ризотто с белыми грибами и пармезаном.,В сотейнике на оливковом масле обжариваем лук-...,"Рис: 70 г, Оливковое масло: 20 мл, Сливочное м...","Калорий 418 ккал, Белки 4 грамм, Жиры 38 грамм...",Итальянская кухня,35 минут,https://eda.ru/recepty/rizotto/rizotto-s-belym...,Ризотто
20134,Ризотто с соусом песто и кедровыми орешками,Для приготовления соуса песто смешать листья б...,"Оливковое масло: 9 столовых ложек, Жемчужный л...","Калорий 906 ккал, Белки 16 грамм, Жиры 62 грам...",Итальянская кухня,25 минут,https://eda.ru/recepty/rizotto/rizotto-s-souso...,Ризотто
20114,Ванильное ризотто с арахисом,"Поставить молоко греться на плиту, добавив туд...","Молоко: 600 мл, Рис для ризотто: 100 г, Ваниль...","Калорий 1027 ккал, Белки 33 грамм, Жиры 39 гра...",Каши на молоке,40 минут,https://eda.ru/recepty/rizotto/vanilnoe-rizott...,Ризотто
20165,Веганское ризотто со свеклой,Готовить присыпку для ризотто. Обжарить на ско...,"Рис: 320 г, Овощной бульон: 600 мл, Чеснок: 1 ...","Калорий 2175 ккал, Белки 58 грамм, Жиры 34 гра...",Европейская кухня,25 минут,https://eda.ru/recepty/rizotto/veganskoe-rizot...,Ризотто


Пятая рекомендация

In [62]:
text = "Блинчики с вараньем"
rating = get_recommendation(text, vectorizer, items_vec)
df_items.iloc[rating].head()

,name,text,ingredient,energy,time_cook,type_kitchen,link,label
17821,Картофельные блинчики с йогуртом и зеленью,"Картофель вымыть, почистить и натереть на круп...","Куриное яйцо: 2 штуки, Пшеничная мука: 3 столо...","Калорий 282 ккал, Белки 10 грамм, Жиры 15 грам...",Европейская кухня,35 минут,https://eda.ru/recepty/osnovnye-blyuda/kartofe...,Основное блюдо
5648,"Блинчики с ветчиной, яйцами и сыром в панировке",Для приготовления теста: муку смешать с двумя ...,"Пшеничная мука: 200 г, Куриное яйцо: 8 штук, М...","Калорий 1069 ккал, Белки 61 грамм, Жиры 66 гра...",Русская кухня,25 минут,https://eda.ru/recepty/zakuski/blinchiki-s-vet...,Закуска
35179,Блинчики с ветчиной под хрустящей сырной корочкой,Растопите 2 столовые ложки масла и дайте ему н...,"Сливочное масло: 150 г, Пшеничная мука: 125 г,...","Калорий 494 ккал, Белки 19 грамм, Жиры 42 грам...",Французская кухня,1 час 20 минут,https://eda.ru/recepty/vypechka-deserty/blinch...,Выпечка
16871,Блинчики с начинкой в панировке,"Муку просеиваем, смешиваем с молоком, 4 яйцами...","Молоко: 750 мл, Куриное яйцо: 10 штук, Пшеничн...","Калорий 621 ккал, Белки 35 грамм, Жиры 38 грам...",Русская кухня,1 час 40 минут,https://eda.ru/recepty/osnovnye-blyuda/blinchi...,Основное блюдо
33447,Блинчики с заварным кремом,"Сделать тесто из 200 г муки, 500 мл молока, 2 ...","Молоко: 1,1 л, Пшеничная мука: 250 г, Куриное ...","Калорий 965 ккал, Белки 22 грамм, Жиры 41 грам...",Французская кухня,1 час,https://eda.ru/recepty/vypechka-deserty/blinch...,Выпечка
